# Toy experiments

This notebook displays regions obtained by different multi-output conformal methods

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import matplotlib.pyplot as plt
import torch
from tqdm import tqdm

from moc.models.oracle.oracle_model import OracleModel
from moc.models.mqf2.lightning_module import MQF2LightningModule
from moc.models.mixture.mixture_model import MixtureLightningModule
from moc.models.trainers.lightning_trainer import get_lightning_trainer
from moc.models.trainers.default_trainer import DefaultTrainer
from moc.configs.config import get_config
from moc.utils.general import seed_everything, savefig
from moc.utils.run_config import RunConfig
from moc.analysis.dataframes import load_datamodule
from moc.analysis.plot_2d_vs_1d import plot_2D_region_vs_1D_per_method, plot_2D_region_vs_1D_per_model, plot_2D_region_vs_1D_for_HDR_CP_per_n_samples
from moc.analysis.plot_2d import plot_2D_region_per_method, plot_2D_regions_by_x_and_tau
from moc.datamodules.toy_datamodule import UnimodalHeteroscedastic, BimodalHeteroscedastic

name = 'toy'
path = Path('results') / name
path.mkdir(exist_ok=True, parents=True)
seed_everything(0)

In [2]:
config = get_config()
config.device = 'cuda' # 'cuda' or 'cpu'

## Unimodal dataset

### Model training

In [ ]:
dataset = 'unimodal_heteroscedastic'
rc = RunConfig(config, 'toy_2dim', dataset, 0, hparams={})
datamodule = load_datamodule(rc)
oracle_model = OracleModel()
trainer = DefaultTrainer(rc)
trainer.fit(oracle_model, datamodule)
p, q = datamodule.input_dim, datamodule.output_dim
mqf2_model = MQF2LightningModule(p, q)
trainer = get_lightning_trainer(rc)
trainer.fit(mqf2_model, datamodule)
mqf2_model.to(config.device);

### 3D plots of 2D output regions vs 1D input

In [ ]:
# List of methods with default hyperparameters
methods = ['M-CP', 'DR-CP', 'HDR-CP', 'PCP', 'HD-PCP', 'C-PCP', 'L-CP']
hparams_list = [{'posthoc_method': method} for method in methods]

torch.manual_seed(0)
plot_2D_region_vs_1D_per_method(hparams_list, datamodule, config, oracle_model, mqf2_model, path=path / 'contours' / f'{dataset}_per_method.pdf', grid_side=300, nrows=2)

### 2D plots of 2D output regions

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3.7, 3.7))
plot_2D_region_per_method(ax, 1, 0.8, datamodule, oracle_model, mqf2_model, grid_side=800, custom_xlim=(-2.8,2.8), custom_ylim=(-2.8,2.8))
ax.legend(loc='upper center', bbox_to_anchor=(0.46, 1.22), ncol=3, prop={'family': plt.rcParams['font.family']})
savefig(path / 'contours' / f'{dataset}_2d_x_1.pdf')

In [ ]:
plot_2D_regions_by_x_and_tau(datamodule, oracle_model, mqf2_model, path=path / 'contours' / f'{dataset}_conf_slices.pdf', grid_side=100)

## Bimodal dataset

### Model training

In [ ]:
dataset = 'bimodal_heteroscedastic'
rc = RunConfig(config, 'toy_2dim', dataset, 0, hparams={})
datamodule = load_datamodule(rc)
oracle_model = OracleModel()
trainer = DefaultTrainer(rc)
trainer.fit(oracle_model, datamodule)
p, q = datamodule.input_dim, datamodule.output_dim
mqf2_model = MQF2LightningModule(p, q)
trainer = get_lightning_trainer(rc)
trainer.fit(mqf2_model, datamodule)
mqf2_model.to(config.device);

### 3D plots of 2D output regions vs 1D input

In [ ]:
torch.manual_seed(0)
plot_2D_region_vs_1D_per_method(hparams_list, datamodule, config, oracle_model, mqf2_model, path=path / 'contours' / f'{dataset}_per_method.pdf', grid_side=300, nrows=2)

### 2D plots of 2D output regions

In [ ]:
plot_2D_regions_by_x_and_tau(datamodule, oracle_model, mqf2_model, path=path / 'contours' / f'{dataset}_conf_slices.pdf', grid_side=100)

## More complex toy dataset

### Model training

In [ ]:
dataset = 'toy_del_barrio'
rc = RunConfig(config, 'toy_2dim', dataset, 0, hparams={})
datamodule = load_datamodule(rc)
p, q = datamodule.input_dim, datamodule.output_dim
mqf2_model = MQF2LightningModule(p, q, icnn_hidden_size=100, icnn_num_layers=3)
#mqf2_model = MixtureLightningModule(mixture_size=1, hidden_size=200, num_layers=4)
trainer = get_lightning_trainer(rc)
trainer.fit(mqf2_model, datamodule)
mqf2_model.to(config.device);

### 3D plots of 2D output regions vs 1D input

In [ ]:
torch.manual_seed(0)
plot_2D_region_vs_1D_per_method(hparams_list, datamodule, config, None, mqf2_model, path=path / 'contours' / f'{dataset}_per_method.pdf', grid_side=300, nrows=2)

## Models comparison on a bimodal dataset

### Models training

In [ ]:
from moc.models.train import models, trainers

dataset = 'bimodal_heteroscedastic'
rc = RunConfig(config, 'toy_2dim', dataset, 0, hparams={})
datamodule = load_datamodule(rc)
p, q = datamodule.input_dim, datamodule.output_dim

models_order = ['Mixture', 'DRF-KDE', 'MQF2']
trained_models = {}
for model_name in models_order:
    model = models[model_name](p, q)
    trainer = trainers[model_name](rc=rc)
    trainer.fit(model=model, datamodule=datamodule)
    print(f'Finished training {rc.summary_str()}')
    model.to(rc.config.device)
    trained_models[model_name] = model

### 3D plots of 2D output regions vs 1D input

In [ ]:
oracle_model = OracleModel()
trainer = DefaultTrainer(rc)
trainer.fit(oracle_model, datamodule)

torch.manual_seed(0)
plot_2D_region_vs_1D_per_model(datamodule, config, oracle_model, trained_models, path=path / 'contours' / f'{dataset}_per_model.pdf', grid_side=300)

## Varying number of samples

### 3D plots of 2D output regions vs 1D input for HDR-CP with varying number of samples

In [ ]:
torch.manual_seed(0)
plot_2D_region_vs_1D_for_HDR_CP_per_n_samples(datamodule, config, oracle_model, path=path / 'contours' / f'{dataset}_HDR_CP_per_n_samples.pdf', grid_side=300)